In [1]:
#CARGAMOS LIBRERIAS
import pandas as pd
import json

In [2]:

# Lista para almacenar los objetos JSON individuales
user_reviews = []

# Abre el archivo JSON y carga su contenido en un diccionario
with open('../../jsons/output_steam_games.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        try:
            json_data = json.loads(line)
            user_reviews.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON en línea: {e}")

# Crear el DataFrame anidado
df_games_anidado = pd.DataFrame(user_reviews)

In [3]:
df_games_anidado.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# Divido el DataFrame en dos DataFrames a partir del índice 88310, ya que los datos pueden ser utiles
df_items2 = df_games_anidado.loc[:88309]
df_games = df_games_anidado.loc[88310:]

In [50]:
df_games.head(2)

,genres,app_name,release_date,specs,item_id,precio
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],761140,4.99
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",643980,0.00


In [10]:
#Elimino las columnas que no voy a usar
df_games=df_games.reset_index()
eliminar_columnas=['publisher','title','url','reviews_url','early_access','developer','tags','index']

df_games=df_games.drop(eliminar_columnas, axis=1)

In [11]:
# función que convierte un valor a float y devolverá 0 si falla
def convert_to_float(value):
    try:
        return float(value)
    except (ValueError, TypeError):
        return 0.0

# Aplica la función a la columna 'price' del DataFrame
df_games['precio'] = df_games['price'].apply(lambda x: convert_to_float(x))
df_games['precio'] = df_games['precio'].fillna(0)

In [12]:
#saco la columna price y dejo la columna id  como item_id para que todos los df tengan el mismo nombre de id
df_games.rename(columns={'id': 'item_id'}, inplace=True)

df_games=df_games.drop('price', axis=1)

In [13]:
#vemos como quedo el df
df_games.head(4)

,genres,app_name,release_date,specs,item_id,precio
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],761140,4.99
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",643980,0.00
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017-07-24,"[Single-player, Multi-player, Online Multi-Pla...",670290,0.00
3,"[Action, Adventure, Casual]",弹炸人2222,2017-12-07,[Single-player],767400,0.99


In [14]:
#convertir a fecha los valores por año dejando como NaN los errores
df_games['release_date'] = pd.to_datetime(df_games['release_date'], errors='coerce', format='%Y-%m-%d')
# Filtrar el DataFrame original para mantener solo las filas que tienen género y año no nulos
df_games = df_games.dropna(subset=['genres','release_date'], how='all')
df_games = df_games.dropna(subset=['item_id'])
# Asignar una fecha predeterminada a los valores vacios
df_games['release_date'].fillna(pd.Timestamp('1900'), inplace=True)
df_games['release_date']=df_games['release_date'].dt.year


In [15]:
df_games.head(4)

,genres,app_name,release_date,specs,item_id,precio
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018,[Single-player],761140,4.99
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018,"[Single-player, Multi-player, Online Multi-Pla...",643980,0.00
2,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,2017,"[Single-player, Multi-player, Online Multi-Pla...",670290,0.00
3,"[Action, Adventure, Casual]",弹炸人2222,2017,[Single-player],767400,0.99


In [16]:

# Crear un DataFrame con géneros como variables dummy
genres_games_df = df_games[['item_id']].copy()
for index, row in df_games.iterrows():
    if isinstance(row['genres'], list):
        for genre in row['genres']:
            genres_games_df.loc[index, genre] = 1

# Llenar NaN con 0 en el DataFrame de géneros
genres_games_df = genres_games_df.fillna(0)

# Ver el DataFrame resultante
genres_games_df.head()

,item_id,Action,Casual,Indie,Simulation,Strategy,Free to Play,RPG,Sports,Adventure,...,Animation &amp; Modeling,Video Production,Utilities,Web Publishing,Education,Software Training,Design &amp; Illustration,Audio Production,Photo Editing,Accounting
0,761140,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,643980,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,670290,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,767400,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,772540,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# Crear un DataFrame con specs como variables dummy
specs_games_df = df_games[['item_id']].copy()
for index, row in df_games.iterrows():
    if isinstance(row['specs'], list):
        for genre in row['specs']:
            specs_games_df.loc[index, genre] = 1
 # Llenar NaN con 0 en el DataFrame de specs
specs_games_df = specs_games_df.fillna(0)
 # Ver el DataFrame resultante
specs_games_df.head()

,item_id,Single-player,Multi-player,Online Multi-Player,Cross-Platform Multiplayer,Steam Achievements,Steam Trading Cards,In-App Purchases,Stats,Downloadable Content,...,Captions available,Steam Workshop,Includes level editor,Mods,Mods (require HL2),Game demo,Includes Source SDK,Commentary available,SteamVR Collectibles,Mods (require HL1)
0,761140,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,643980,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,670290,1.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,767400,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,772540,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
#apilcando reduccion de dimensionalidad
# Sumar las columnas y crear un DataFrame
genres_sum_df = genres_games_df.sum().reset_index()
genres_sum_df.columns = ['Genre', 'Count']

# Convertir la columna "Count" a valores numéricos
genres_sum_df['Count'] = pd.to_numeric(genres_sum_df['Count'], errors='coerce')

# Ordenar el DataFrame por la columna "Count" de forma descendente
sorted_genres_df = genres_sum_df.sort_values(by='Count', ascending=False)

# Obtener los 20 géneros más comunes y sus recuentos
top_20_genres = sorted_genres_df.head(20)

# Calcular el porcentaje de aparición de cada género
total_count = top_20_genres['Count'].sum()
top_20_genres['Percentage'] = (top_20_genres['Count'] / total_count) * 100

# Imprimir los resultados
print(top_20_genres)


                        Genre    Count  Percentage
3                       Indie  15858.0   22.188952
1                      Action  11320.0   15.839257
2                      Casual   8282.0   11.588403
9                   Adventure   8242.0   11.532434
5                    Strategy   6957.0    9.734427
4                  Simulation   6699.0    9.373426
7                         RPG   5479.0    7.666368
6                Free to Play   2031.0    2.841831
11               Early Access   1462.0    2.045671
8                      Sports   1257.0    1.758829
12      Massively Multiplayer   1108.0    1.550344
10                     Racing   1083.0    1.515364
19  Design &amp; Illustration    460.0    0.643645
15                  Utilities    340.0    0.475737
16             Web Publishing    268.0    0.374993
13   Animation &amp; Modeling    183.0    0.256059
17                  Education    125.0    0.174903
14           Video Production    116.0    0.162310
18          Software Training  

C:\Users\lgtqa\AppData\Local\Temp\ipykernel_13200\3479459093.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_20_genres['Percentage'] = (top_20_genres['Count'] / total_count) * 100


In [21]:
#apilcando reduccion de dimensionalidad
#Sumar las columnas y crear un DataFrame
specs_sum_df = specs_games_df.sum().reset_index()
specs_sum_df.columns = ['Specs', 'Count']

#Convertir la columna "Count" a valores numéricos
specs_sum_df['Count'] = pd.to_numeric(specs_sum_df['Count'], errors='coerce')

#Ordenar el DataFrame por la columna "Count" de forma descendente
sorted_specs_df = specs_sum_df.sort_values(by='Count', ascending=False)

#Obtener los 20 géneros más comunes y sus recuentos
top_20_specs = sorted_specs_df.head(20)

#Calcular el porcentaje de aparición de cada género
total_count = top_20_specs['Count'].sum()
top_20_specs['Percentage'] = (top_20_specs['Count'] / total_count) * 100

#Imprimir los resultados
print(top_20_specs)

                         Specs    Count  Percentage
1                Single-player  25765.0   20.467581
5           Steam Achievements  15819.0   12.566531
9         Downloadable Content  12658.0   10.055449
6          Steam Trading Cards  11946.0    9.489840
11                 Steam Cloud   9743.0    7.739788
2                 Multi-player   8007.0    6.360719
10     Full controller support   7101.0    5.640997
13  Partial Controller Support   6371.0    5.061089
12          Steam Leaderboards   5769.0    4.582863
20                       Co-op   4211.0    3.345196
15         Shared/Split Screen   3901.0    3.098934
4   Cross-Platform Multiplayer   2459.0    1.953417
8                        Stats   2196.0    1.744491
3          Online Multi-Player   2164.0    1.719070
23              Steam Workshop   1825.0    1.449770
22          Captions available   1612.0    1.280564
24       Includes level editor   1335.0    1.060517
17          Local Multi-Player   1150.0    0.913554
19          

C:\Users\lgtqa\AppData\Local\Temp\ipykernel_13200\3618210682.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_20_specs['Percentage'] = (top_20_specs['Count'] / total_count) * 100


In [22]:
columnas_elegidas = ['item_id', 'Single-player','Multi-player', 'Full controller support',
                       'Partial Controller Support', 'Steam Leaderboards', 'Co-op',
                         'Cross-Platform Multiplayer','Online Multi-Player']

specs_df = specs_games_df[columnas_elegidas]


In [23]:
columnas_elegidas = [
    'item_id', 'Indie', 'Action', 'Casual', 'Adventure', 'Strategy', 'Simulation',
    'RPG', 'Free to Play', 'Early Access', 'Sports', 'Massively Multiplayer',
    'Racing']

# Crear un nuevo DataFrame con las columnas seleccionadas
genres_df = genres_games_df[columnas_elegidas]

In [65]:
specs_df.head(2)

,item_id,Single-player,Multi-player,Full controller support,Partial Controller Support,Steam Leaderboards,Co-op,Cross-Platform Multiplayer,Online Multi-Player
0,761140,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,643980,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0


In [64]:
genres_df.head(2)

,item_id,Indie,Action,Casual,Adventure,Strategy,Simulation,RPG,Free to Play,Early Access,Sports,Massively Multiplayer,Racing
0,761140,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,643980,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0


In [63]:
df_games.head(2)

,genres,app_name,release_date,specs,item_id,precio
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],761140,4.99
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",643980,0.00


In [24]:
# guardo el archivo con solo los generos en un csv
nombre_archivo = "../../fastapi/games_genres.csv"

genres_df.to_csv(nombre_archivo, index=False) 

In [25]:
#guardo el archivo con solo los generos en un csv
nombre_archivo = "../../fastapi/games_specs.csv"

specs_df.to_csv(nombre_archivo, index=False) 

In [153]:
# # guardo el archivo con solo los juegos en un csv
# nombre_archivo = "../../fastapi/games.csv"

# df_games.to_csv(nombre_archivo, index=False) 

In [68]:
df_games.head(2)

,genres,app_name,release_date,specs,item_id,precio
0,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,2018-01-04,[Single-player],761140,4.99
1,"[Free to Play, Indie, RPG, Strategy]",Ironbound,2018-01-04,"[Single-player, Multi-player, Online Multi-Pla...",643980,0.00


In [26]:
df_games=df_games.drop(['genres','specs'], axis=1)

In [27]:
df_games.head(2)

,app_name,release_date,item_id,precio
0,Lost Summoner Kitty,2018,761140,4.99
1,Ironbound,2018,643980,0.00


In [32]:
# guardo el archivo con solo los juegos en un csv
nombre_archivo = "../../fastapi/games.csv"

df_games.to_csv(nombre_archivo, index=False) 